In [1]:
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [2]:
# считываем данные, пропустим данные, где количество правильных ответов больше одного
test = pd.read_csv("task2_lemmas_test")
train = pd.read_csv("task2_lemmas_train", error_bad_lines=False, warn_bad_lines=False)
print(len(test), len(train))

(29661, 116477)


In [3]:
train.head()

,Id,X,y
0,1,vergognerete,vergognare+V
1,2,amnistiavate,amnistiare+V
2,3,menomazione,menomazione+N
3,4,sfaldavamo,sfaldare+V
4,5,sfodererei,sfoderare+V


Нужно разделить столбец Y на нормальную форму и часть речи

In [4]:
train['norm'] = train['y'].apply(lambda x: x[:-2])
train['part'] = train['y'].apply(lambda x: x[-1:])
train = train.drop(['Id', 'y'], axis=1)
train.head()

,X,norm,part
0,vergognerete,vergognare,V
1,amnistiavate,amnistiare,V
2,menomazione,menomazione,N
3,sfaldavamo,sfaldare,V
4,sfodererei,sfoderare,V


Сначала будем предсказывать часть речи, используем для этого n-граммы как в предыдущем контесте

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
xtrain, xtest = train_test_split(train.drop('norm', axis=1))

In [7]:
%%time
vectorizer = CountVectorizer(ngram_range=(2,8), analyzer='char_wb', max_df=0.84)

# извлекаем признаки из данных, признаками будут n-граммы и сразу же делаем матрицу признаков
train_matrix = vectorizer.fit_transform(xtrain['X'])

Wall time: 10 s


In [ ]:
# получили разреженную матрицу
train_matrix

<87357x430069 sparse matrix of type '<type 'numpy.int64'>'
	with 5162284 stored elements in Compressed Sparse Row format>

In [ ]:
# теперь обучаем нашу модель на извлечённых признаках, признаков много, поэтому используем регуляризацию l1, которая
# осуществляет отбор признаков
model = LogisticRegression(penalty='l1', n_jobs=-1, random_state=13)
model.fit(train_matrix, xtrain['part'])
predictions = model.predict(vectorizer.transform(xtest['X']))

In [ ]:
print(predictions[:5])

In [ ]:
print(accuracy_score(predictions, xtest['part']))

Нужно определить нормальную форму. Нужно отсечь приставку и окончание от слова и добавить другие приставку и окончание

Посмотрим, у скольких слов не совпадают приставки и у скольких не совпадают окончания

In [ ]:
pref = 0
suf = 0
for i in range(len(train)):
    if train.iloc[i, 0][:2] != train.iloc[i, 1][:2]:
        pref += 1
    if train.iloc[i, 0][-2:] != train.iloc[i, 1][-2:]:
        suf += 1
print(pref, suf, len(train))

Слов с несовпадающими приставками очень мало, не будем обращать на них внимание. Пусть один классификатор определяет, сколько букв нужно отсечь от слова, а другой - какое окончание нужно сделать

In [5]:
# возвращает количество букв, которые нужно отсечь от слова
def cut_suf(word, norm):
    if word == norm:
        return 0
    count = 1
    while word[count] == norm[count] and count < len(word)-1 and count < len(norm)-1:
        count += 1
    return len(word) - count

In [6]:
# возвращает окончание, которое нужно приписать к слову
def concat_suf(word, norm):
    pref = len(word) - cut_suf(word, norm)
    return norm[pref:]

In [7]:
to_cut = [cut_suf(w1, w2) for w1, w2 in zip(train['X'], train['norm'])]
ssuf = [concat_suf(w1, w2) for w1, w2 in zip(train['X'], train['norm'])]
train['to_cut'] = to_cut
train['suf'] = ssuf
train.head()

,X,norm,part,to_cut,suf
0,vergognerete,vergognare,V,5,are
1,amnistiavate,amnistiare,V,4,re
2,menomazione,menomazione,N,0,
3,sfaldavamo,sfaldare,V,4,re
4,sfodererei,sfoderare,V,4,are


In [8]:
# смотрим, сколько различных суффиксов получилось
print(len(np.unique(ssuf)))

83


Обучаем классификаторы для определения того, сколько букв отсечь

In [ ]:
xtrain, xtest = train_test_split(train.drop(['norm', 'part', 'suf'], axis=1))

In [ ]:
%%time
vectorizer = CountVectorizer(ngram_range=(3,5), analyzer='char_wb', max_df=0.84)
train_matrix = vectorizer.fit_transform(xtrain['X'])

In [ ]:
model = LogisticRegression(penalty='l1', n_jobs=-1, random_state=13)
model.fit(train_matrix, xtrain['to_cut'])
predictions = model.predict(vectorizer.transform(xtest['X']))

In [ ]:
print(accuracy_score(predictions, xtest['to_cut']))

Для суффиксов скорее всего тоже будет работать более менее хорошо, попробуем сделать посылку

# Make a submission!

In [9]:
test.head()

,Id,X
0,1,gettonan
1,2,incidentali
2,3,involtino
3,4,lievi
4,5,comunistizzasse


In [10]:
train.tail()

,X,norm,part,to_cut,suf
116472,posereste,posare,V,6,are
116473,cogestiste,cogestire,V,3,re
116474,autocorreggerebbero,autocorreggere,V,6,e
116475,gorgogliassimo,gorgogliare,V,5,re
116476,desecretaste,desecretare,V,3,re


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
%%time
vectorizer = CountVectorizer(ngram_range=(2, 8), analyzer='char_wb', max_df=0.84)
train_matrix = vectorizer.fit_transform(train['X'])
test_matrix = vectorizer.transform(test['X'])

Wall time: 13.2 s


In [33]:
%%time
# для части речи
model = LogisticRegression(penalty='l1', n_jobs=-1, random_state=13)
model.fit(train_matrix, train['part'])
part = model.predict(test_matrix)

Wall time: 29.3 s


In [34]:
%%time
# сколько букв отрезать
model = LogisticRegression(penalty='l1', n_jobs=-1, random_state=42)
model.fit(train_matrix, train['to_cut'])
to_cut = model.predict(test_matrix)

Wall time: 1min 54s


In [35]:
%%time
# какие буквы добавить
model = LogisticRegression(n_jobs=-1, random_state=6)
model.fit(train_matrix, train['suf'])
suf = model.predict(test_matrix)

Wall time: 10min 12s


In [36]:
def cut_word(w, n):
    if n==0:
        return w
    else:
        return w[:-n]

In [37]:
initials = [cut_word(test.iloc[i,1], to_cut[i]) for i in range (len(test))]

In [38]:
initials[10:20]

['cucio',
 'snobba',
 'tesser',
 'coagulare',
 'somatizzar',
 'impoveriment',
 'smunger',
 'abbuff',
 'meraviglia',
 'risucchi']

In [39]:
answer = [initials[i]+suf[i]+'+'+part[i] for i in range(len(test))]

In [40]:
final = pd.DataFrame(data=test['Id'], columns=['Id'])

In [41]:
final['Category'] = answer

In [43]:
final.to_csv('fourth_super_submission.csv', index=False)

# Finish